In [1]:
import rasterio
import numpy as np

def normalize_raster(input_tif, output_tif):
    """
    Reads a TIFF file, normalizes its values to the range [0, 1], and writes to a new TIFF file.
    NaN values are preserved.

    Args:
        input_tif (str): Path to the input TIFF file.
        output_tif (str): Path to save the normalized TIFF file.
    """
    with rasterio.open(input_tif) as src:
        profile = src.profile
        
        # Read the data
        data = src.read(1).astype(np.float32)
        
        # Identify NaN values
        nan_mask = np.isnan(data)
        
        # Normalize excluding NaN values
        min_val = np.nanmin(data)
        max_val = np.nanmax(data)
        if max_val > min_val:
            normalized_data = (data - min_val) / (max_val - min_val)
        else:
            normalized_data = np.zeros_like(data)  # Avoid division by zero case
        
        # Restore NaN values
        normalized_data[nan_mask] = np.nan
        
        # Update profile to maintain original data type and write output
        profile.update(dtype=rasterio.float32)
        
        with rasterio.open(output_tif, 'w', **profile) as dst:
            dst.write(normalized_data, 1)




In [2]:
# Example usage
input_tiff = "/media/data/raw/nico_resize/CA_Elevation.tif"
output_tiff = "/media/data/raw/nico_resize/CA_Elevation2.tif"
normalize_raster(input_tiff, output_tiff)